In [2]:
import torch
import numpy as np
import torch.nn as nn
import math
import os
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import json

In [3]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        #does not pass all input words with eachother. Each word goes through independantly
        #and the output are the embeddings of the word. We want this because we do not 
        #want to concacenate the embeddings to the output nodes.
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) 
        #now takes in all embeddings of each word stretched out
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        self.tokens = {}

    def forward(self, inputs):
        #embeds will be flattened matrix
        embeds = self.embeddings(inputs).view((1, -1))
        #rectified relu to learn embeddings
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        #output is the log probablities of all vocabulary
        return log_probs

    


In [4]:
#file path to credit card csv file
file_path = os.path.join(os.path.expanduser("~"), "Downloads", "mental_health.csv")
orig_dataset = pd.read_csv(file_path) #read csv file as pandas object
orig_dataset = orig_dataset.to_numpy()
print("Dataset size: "+ str(len(orig_dataset)))
dataset = orig_dataset[0:100] #use part of the dataset
print("Train Dataset size: "+ str(len(dataset)))
minFreq = {} #word must appear n times to be added to dictionary
dictionary = {} #relevant words in the dicationary
index = 3
for example in range(len(dataset)):
    for cont_response in range(2):
        if type(dataset[example][cont_response]) == float: #NaN values
            continue
        for word in dataset[example][cont_response].split():
            if word not in minFreq:
                minFreq[word]=1
            else:
                if minFreq[word]==3: #word needs to appear
                    dictionary[word] = index
                    index+=1
                minFreq[word]+=1

print( "Total Dictionary Size: 10,489")
print("Training Dictionary Size: " + str(index))

CONTEXT_SIZE = 3 #look 3 words back to predict current word
EMBEDDING_DIM = 252 #total embeddings for each word
all_ngrams = [] #ngram setup -> [(['through', 'going', "I'm"], 'some')]
for example in range(len(dataset)): 
    for cont_response in range(2): #context than response
        if type(dataset[example][cont_response]) == float: #NaN values
            continue
        cur_Sentence = dataset[example][cont_response].split() #seperate by word
        ngrams = [ #[(['through', 'going', "I'm"], 'some')]
            ([cur_Sentence[i - j - 1] for j in range(CONTEXT_SIZE)],cur_Sentence[i])
            for i in range(CONTEXT_SIZE, len(cur_Sentence))
            ]
        #append the grams to all_ngrams
        for i in ngrams:
            all_ngrams.append(i) 
loss_function = nn.NLLLoss() #loss layer
model = NGramLanguageModeler(index, EMBEDDING_DIM, CONTEXT_SIZE) #intialize Ngram model
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.tokens = dictionary
for epoch in range(10):
    total_loss = 0
    print("Epoch: "+ str(epoch))
    maxFreq = 3 #max number of times a word can be trained
    #dictionary to keep track of times word is trained. Will skip if words have been trained maxFreq times
    maxFreqDict = {}
    for context, target in all_ngrams:
        #if unknown word, just don't train
        if context[0] not in dictionary or context[1] not in dictionary or context[2] not in dictionary:
                continue
        if target not in dictionary:
                continue
        #add context words if not found in dict
        if context[0] not in maxFreqDict:
            maxFreqDict[context[0]] = 1
        if context[1] not in maxFreqDict:
            maxFreqDict[context[1]] = 1
        #if both words have been trained equal to or more than maxFreq times, continue
        #already has been trained enough
        if maxFreqDict[context[0]] >= maxFreq and maxFreqDict[context[1]] >= maxFreq:
            continue
        #update how many times the context words have been trained
        maxFreqDict[context[0]]+=1
        maxFreqDict[context[1]]+=1
            
        #turn each word to an integer and wrapped in tensor so pass as an input to the model
        context_idxs = torch.tensor([dictionary[w] for w in context], dtype=torch.long)

        #zero out gradients cause it accumulates
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        
        
            

        #apply the loss function to the log probabilties with the correct target word
        loss = loss_function(log_probs, torch.tensor([dictionary[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(f"Total_Loss: {total_loss}")





Dataset size: 3512
Train Dataset size: 100
Total Dictionary Size: 10,489
Training Dictionary Size: 586
Epoch: 0
Total_Loss: 5896.734264612198
Epoch: 1
Total_Loss: 4402.660709857941
Epoch: 2
Total_Loss: 3560.378431379795
Epoch: 3
Total_Loss: 2984.954965353012
Epoch: 4
Total_Loss: 2452.747850239277
Epoch: 5
Total_Loss: 1923.3361112773418
Epoch: 6
Total_Loss: 1413.2404913902283
Epoch: 7
Total_Loss: 969.5563013330102
Epoch: 8
Total_Loss: 626.8774465024471
Epoch: 9
Total_Loss: 390.8301209397614


In [5]:
torch.save(model.state_dict(), "embedding_model")

In [8]:
print(model.embeddings(torch.tensor([0])))

tensor([[-0.8661, -1.8521,  0.0153, -0.2692, -1.5972, -1.1300, -0.3171, -1.6196,
          0.9209, -0.1862,  0.7124, -0.2338,  0.6138, -1.5788,  1.1109, -0.1894,
         -1.3766, -0.1649,  0.4087,  2.5591,  1.9314,  0.6390,  0.0225, -0.3324,
          0.2382, -0.8647,  0.7637,  0.8638, -0.5354, -0.8867,  1.1967, -0.3811,
          1.1656, -0.0634, -0.0229,  0.1177, -0.3202,  0.7285, -1.0219, -0.0105,
         -1.9199,  2.0788, -0.2297,  0.2163, -1.2744, -0.2721, -0.3612,  0.6907,
          0.0296,  1.4764,  1.1057, -0.3567,  0.4677,  0.5065, -1.9349, -0.3043,
          1.2253,  0.2794, -0.1066, -0.1835, -0.6063,  1.0302,  0.9271, -0.4087,
          0.4247,  1.1284, -0.8166,  1.1268, -1.1659,  0.6538,  1.1584, -1.0005,
         -0.3316, -0.7803, -0.5708,  0.3823, -0.1890,  0.6133,  0.3005, -0.6336,
         -0.8436, -0.1964, -0.4177,  0.3837, -0.0891, -0.9741,  1.9166,  0.4739,
          0.7113,  0.4243,  1.3714,  1.3631, -0.2424,  0.4901, -0.8220,  1.9681,
         -0.8166, -1.0336, -

In [22]:
json.dump(model.tokens, open("tokens.txt",'w'))